# Database

In [1]:
import pandas as pd

class LocalDatabase:
  def __init__(self, cpi_dataset="CPI_for_urban_consumers_all_items_US_city_average.csv", m2_dataset="M2.csv", ccpi_dataset="CPILFESL.csv"):
    self.cpi = pd.read_csv(f"/Users/anoop/Downloads/project_2_data/{cpi_dataset}")  
    self.m2 = pd.read_csv(f"/Users/anoop/Downloads/project_2_data/{m2_dataset}")
    self.ccpi = pd.read_csv(f"/Users/anoop/Downloads/project_2_data/{ccpi_dataset}")
    self.df = None  # Placeholder for the processed DataFrame

  def load_data(self):
    # If the DataFrame is already processed, return it directly
    if self.df is not None:
        return self.df
      
    # Convert 'observation_date 'to datetime and assign to 'Date'
    self.cpi["Date"] = pd.to_datetime(self.cpi["observation_date"]) 
    self.m2["Date"] = pd.to_datetime(self.m2["observation_date"])
    self.ccpi["Date"] = pd.to_datetime(self.ccpi["observation_date"])

    # Drop the 'observation_date' column
    self.cpi.drop(columns=["observation_date"], inplace=True)
    self.m2.drop(columns=["observation_date"], inplace=True)
    self.ccpi.drop(columns=["observation_date"], inplace=True)

    # Rename columns
    self.cpi.rename(columns={"CPIAUCSL": "CPI"}, inplace=True)
    self.m2.rename(columns={"M2SL": "M2"}, inplace=True)
    self.ccpi.rename(columns={"CPILFESL": "CCPI"}, inplace=True)
    
    # Set 'Date' as the index for both dataframes
    self.cpi.set_index("Date", inplace=True)
    self.m2.set_index("Date", inplace=True)
    self.ccpi.set_index("Date", inplace=True)
    
    # Concatenate the two DataFrames along the columns
    self.df = pd.concat([self.cpi, self.m2, self.ccpi], axis=1)

    return self.df

# Business

In [2]:
import joblib
import pandas as pd
#from database import LocalDatabase 
import plotly.express as px


class Model:
  def __init__(self, cpi_model="cpi_model.pkl", m2_model="m2_model.pkl", ccpi_model="ccpi_model.pkl"):
    self.cpi_model = joblib.load("cpi_model.pkl")
    self.m2_model = joblib.load("m2_model.pkl")
    self.ccpi_model = joblib.load("ccpi_model.pkl")
    self.db = LocalDatabase()

  def predict(self, years = range(1,11), result="fig_cpi"):
    # Load training data from database
    df = self.db.load_data()
    
    # Initialize M2 value and corresponding year
    m2 = 20810.3  # M2 for 2023
    corresponding_year = 2023
    
    # Lists to store predicted M2s and corresponding years
    predicted_m2s = []
    corresponding_years = []
    
    for year in years:
        # Prepare the data for prediction
        #X = np.array([[m2]])
        X = pd.DataFrame({"M2_L1": [m2]})
    
        # Make prediction using the trained model
        predicted_m2 = self.m2_model.predict(X)
    
        # Update M2 and corresponding year for the next iteration
        m2 = predicted_m2[0]  # Update the value of m2 for the next year
        corresponding_year += 1
    
        # Append the results to the respective lists
        predicted_m2s.append(predicted_m2[0])  # Store as a scalar
        corresponding_years.append(corresponding_year)

    # Create a DataFrame to store forecasted results
    df_predicted_m2 = pd.DataFrame({
      "Year": corresponding_years,
      "M2": predicted_m2s
    })
    
    # Convert Year to pandas datetime and set as index
    df_predicted_m2["Year"] = pd.to_datetime(df_predicted_m2["Year"], format="%Y")
    df_predicted_m2.set_index("Year", inplace=True)
    
    # Predict CPI
    predicted_cpis = self.cpi_model.predict(df_predicted_m2)  

    # Predict CCPI
    predicted_ccpis = self.ccpi_model.predict(df_predicted_m2)  
    
    # Create a DataFrame with the predicted CPI values and the corresponding datetime index
    df_predicted_cpi = pd.DataFrame(
        predicted_cpis, 
        columns=["CPI"],  # Name of the column with predictions
        index=df_predicted_m2.index  # Use the same datetime index from df_predicted_m2
    )

    # Create a DataFrame with the predicted CCPI values and the corresponding datetime index
    df_predicted_ccpi = pd.DataFrame(
        predicted_ccpis, 
        columns=["CCPI"],  # Name of the column with predictions
        index=df_predicted_m2.index  # Use the same datetime index from df_predicted_m2
    )
    
    # Drop `M2` from `df`
    df = df.drop(columns=["M2"])
    
    #  Concatenate predicted CPI and CCPI dataframes horizontally
    df_predicted_cpis = pd.concat([df_predicted_cpi, df_predicted_ccpi], axis=1)

    #Align the index of df_predicted_cpis with df_predicted_m2
    df_predicted_cpis.index = df_predicted_m2.index

    # Concatenate the original dataframe with the predicted values
    df = pd.concat([df, df_predicted_cpis])

    # Calculate Percent Change Year Ago for CPI and CCPI
    df["CPI_PC"] = df["CPI"].pct_change() * 100
    df["CCPI_PC"] = df["CCPI"].pct_change() * 100
    
    # Drop rows with NaN values resulting from percent change calculation
    df = df.dropna()

    # Add a 'Label' column
    df["Label"] = ["Actual" if idx.year <= 2023 else "Predicted" for idx in df.index]

    # Figure for CPI
    fig_cpi = px.line(
        df,
        x=df.index,
        y="CPI",
        title=(
            "Consumer Price Index for All Urban Consumers: "
            "All Items in U.S. City Average (1961 - 2033)"
        ),
        labels={
            "CPI": "CPI (Index 1982-1984=100)",
            "index": "Year"
        },
        template="plotly_white"
    )

    # Figure for CCPI
    fig_ccpi = px.line(
        df,
        x=df.index,
        y="CCPI",
        title=(
            "Consumer Price Index for All Urban Consumers: "
            "All Items Less Food and Energy in U.S. City Average (1961 - 2033)"
        ),
        labels={
            "CPI": "CPI (Index 1982-1984=100)",
            "index": "Year"
        },
        template="plotly_white"
    )

    # Figure for CPI percent change
    fig_cpi_pc = px.line(
        df,
        x=df.index,
        y="CPI_PC",
        title="Percentage Change a Year Ago in CPI: All Items in U.S. City Average (1961 - 2033)",
        labels={"CPI_PC": "% Change a Year Ago", "index": "Year"},
        template="plotly_white"
    )

    # Figure for CCPI percent change
    fig_ccpi_pc = px.line(
        df,
        x=df.index,
        y="CCPI_PC",
        title="Percentage Change a Year Ago in CPI: All Items Less Food and Energy in U.S. City Average (1961 - 2033)",
        labels={"CPI_PC": "% Change a Year Ago", "index": "Year"},
        template="plotly_white"
    )

    if result == "fig_cpi":
        fig = fig_cpi
    elif result == "fig_cpi_pct_chg":
        fig = fig_cpi_pc
    elif result == "fig_ccpi":
        fig = fig_ccpi
    elif result == "fig_ccpi_pct_chg":
        fig = fig_ccpi_pc
    elif result == "dataframe":
        fig = df
    
    return fig

In [3]:
model = Model()

In [8]:
df = model.predict(result="dataframe")

In [16]:
# Filter for predicted values (2024–2033)
df_predicted = df[df["Label"] == "Predicted"]

# Calculate the mean CPI and CCPI for the predicted values
mean_cpi = df_predicted["CPI"].mean()
mean_ccpi = df_predicted["CCPI"].mean()

# Get the actual CPI and CCPI values for 2023
actual_cpi_2023 = df.loc["2023-01-01", "CPI"]
actual_ccpi_2023 = df.loc["2023-01-01", "CCPI"]

# Calculate percent change from 2023 actual to predicted mean
percent_change_cpi = ((mean_cpi - actual_cpi_2023) / actual_cpi_2023) * 100
percent_change_ccpi = ((mean_ccpi - actual_ccpi_2023) / actual_ccpi_2023) * 100

# Print the results
print(f"Mean CPI (2024-2033): {mean_cpi:.2f}")
print(f"Mean CCPI (2024-2033): {mean_ccpi:.2f}")
print(f"Percent Change in CPI (2023 to Mean): {percent_change_cpi:.2f}%")
print(f"Percent Change in CCPI (2023 to Mean): {percent_change_ccpi:.2f}%")

Mean CPI (2024-2033): 462.20
Mean CCPI (2024-2033): 470.86
Percent Change in CPI (2023 to Mean): 51.69%
Percent Change in CCPI (2023 to Mean): 52.69%


In [22]:
mean_cpi.round(2)

462.2

In [20]:
# Calculate Purchasing Power for 2023 and 2033
pp_cpi_2023 = 100 / df.loc["2023-01-01", "CPI"]
pp_cpi_2033 = 100 / df.loc["2033-01-01", "CPI"]

pp_ccpi_2023 = 100 / df.loc["2023-01-01", "CCPI"]
pp_ccpi_2033 = 100 / df.loc["2033-01-01", "CCPI"]

# Calculate Percent Change in Purchasing Power
pp_cpi_change = ((pp_cpi_2033 - pp_cpi_2023) / pp_cpi_2023) * 100
pp_ccpi_change = ((pp_ccpi_2033 - pp_ccpi_2023) / pp_ccpi_2023) * 100

# Display metrics
print(f"Purchasing Power (CPI) in 2033: {pp_cpi_2033:.2f}")
print(f"Percent Change (CPI) from 2023 to 2033: {pp_cpi_change:.2f}%")

print(f"Purchasing Power (CCPI) in 2033: {pp_ccpi_2033:.2f}")
print(f"Percent Change (CCPI) from 2023 to 2033: {pp_ccpi_change:.2f}%")

Purchasing Power (CPI) in 2033: 0.17
Percent Change (CPI) from 2023 to 2033: -47.11%
Purchasing Power (CCPI) in 2033: 0.17
Percent Change (CCPI) from 2023 to 2033: -47.42%
